In [10]:
from textblob import TextBlob
import math
from collections import Counter, defaultdict
import tweepy as tw
import json
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
%matplotlib inline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [11]:
api_key = "HAcIeikl6eRheQP1oKYGDwPx0"
api_secret = "gEWmqG7QVXzudnviUXDo98L2UKGYg9PNqy1bAR1geClkHjqBGk"
access_token = "1481095074465579008-1pApYlU4HRJWpbAXcbdVEVGzdACuEz"
access_token_secret = "htgD81H4mOxXw5MglAljFRHhcfGAuwsNomXVqvGw0Obwe"
auth = tw.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)

In [12]:
number_of_tweets = 300
tweets = []
user_id =[]
created_at = []
keyword_list = ['News', 'Stock'] # , 'Customers', 'Employees', 'Company', 'Price', 'Shareholders', 'Market', 'Soaring', 'Trends'
company = 'Amazon'
final_kw_list = []

# Crawl data from keywords
for i in range (len(keyword_list)):
    kw = str(company + ' ' + keyword_list[i])
    print(kw)
    final_kw_list.append(kw)
    for i in tw.Cursor(api.search, q = kw , lang ="en", since = "2022-01-14",tweet_mode = "extended").items(number_of_tweets):
        tweets.append(i.full_text)
        user_id.append(i.user.id)
        created_at.append(i.created_at)

Amazon News
Amazon Stock


In [13]:
df = pd.DataFrame({"Created_at": created_at,"User_id": user_id,"Tweets":tweets})
df

,Created_at,User_id,Tweets
0,2022-02-07 06:02:21,1023855424632586241,"Amazon and Nike exploring Peloton takeover, re..."
1,2022-02-07 06:02:14,101747128,"Amazon and Nike exploring Peloton takeover, re..."
2,2022-02-07 06:02:09,49404200,"Amazon and Nike exploring Peloton takeover, re..."
3,2022-02-07 06:01:42,1468248876675178502,"RT @katydyers: Some fun news - my first book, ..."
4,2022-02-07 06:00:35,1223569742364938240,"Amazon and Nike exploring Peloton takeover, re..."
...,...,...,...
595,2022-02-07 01:31:04,41909613,RT @DiscussingFilm: 'The Art and Soul of Dune'...
596,2022-02-07 01:31:04,1261001485749452802,RT @DiscussingFilm: 'The Art and Soul of Dune'...
597,2022-02-07 01:30:50,1389773905410379777,RT @DiscussingFilm: 'The Art and Soul of Dune'...
598,2022-02-07 01:30:44,780460754910732288,'The Art and Soul of Dune' is back in stock an...


In [14]:
df = df[~df.Tweets.str.contains("RT")]
df = df.reset_index(drop = True)

In [15]:
def clean_up_tweet(text):
    text = re.sub(r'@[A-Za-z0-9_]+','',text) # Remove mentions @
    text = re.sub(r'#', '', text) # Remove hastag symbol
    text = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', text) # Remove hyper link
    text = re.sub('[^A-Za-z0-9]+', ' ', text) # Remove special characters
    text = re.sub(r'[0-9]+', '', text)
    #Remove stop words
    res = ''
    for word in text.split():
        if word not in stopwords.words('english'):
            res += WordNetLemmatizer().lemmatize(word) + ' '

    return res

In [16]:
df['Tweets'] = df['Tweets'].apply(clean_up_tweet)
df = df.drop_duplicates(subset='Tweets')
df = df.reset_index(drop=True)
df

,Created_at,User_id,Tweets
0,2022-02-07 06:02:21,1023855424632586241,Amazon Nike exploring Peloton takeover report ...
1,2022-02-07 06:00:35,1223569742364938240,Amazon Nike exploring Peloton takeover report ...
2,2022-02-07 06:00:19,1354623932,Thank news article No matter talented someone ...
3,2022-02-07 05:58:59,2719361006,seek money reopen Evaly server Retailer Retail...
4,2022-02-07 05:55:21,842726388469841921,Amazon Nike exploring Peloton takeover report ...
...,...,...,...
209,2022-02-07 01:47:33,1142815161251782663,I think incredibly thoughtful I test regularly...
210,2022-02-07 01:46:40,25656181,Peloton stock surge report Amazon Nike among p...
211,2022-02-07 01:44:35,56600745,Amazon AMZN Stock Price Target From MKM Partners
212,2022-02-07 01:35:45,1306463816226660352,Amazon ASUS ROG Strix Z A Gaming WiFi D LGA AT...


In [17]:
# Create function to get subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# Create function to get polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# Function compute negative, positive
def getAnalysis(score):
    if score < 0:
        return np.int_(0)
    elif score > 0:
        return np.int_(1)
    else:
        return 'Neutral'

df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)
df['Label'] = df['Polarity'].apply(getAnalysis)
df.drop(df.loc[df['Label']=='Neutral'].index, inplace=True)
df = df.reset_index(drop=True)
df

,Created_at,User_id,Tweets,Subjectivity,Polarity,Label
0,2022-02-07 06:00:19,1354623932,Thank news article No matter talented someone ...,0.460417,0.277083,1
1,2022-02-07 05:53:37,1223569742364938240,Check deal Apple iPhone mini iPhone available ...,0.400000,0.400000,1
2,2022-02-07 05:53:31,2732529079,Jeff Bezos net worth surged billion billion je...,0.050000,0.150000,1
3,2022-02-07 05:49:40,1375679610344640517,EXCITING NEWS Liv going continue holding baton,0.800000,0.300000,1
4,2022-02-07 05:48:18,1330105258358530055,airtel making compulsory purchase Amazon subsc...,0.900000,-0.500000,0
...,...,...,...,...,...,...
113,2022-02-07 02:31:04,1006334607946649600,Top Stock Market Moving Headlines From Last We...,0.355556,0.333333,1
114,2022-02-07 02:22:43,14562685,MyPOV fire sale What install base worth darlin...,0.700000,-0.166667,0
115,2022-02-07 02:14:20,2761384656,Do know people put investment capital Googles ...,0.650000,0.350000,1
116,2022-02-07 01:52:43,20110439,amazon stock shift frequently right listing go...,0.433929,0.371429,1


In [ ]:
class Tokenizer:

    def clean(self, text):
        no_html = BeautifulSoup(text).get_text()
        clean = re.sub("[^a-z\s]+", " ", no_html, flags=re.IGNORECASE)
        return re.sub("(\s+)", " ", clean)


    def tokenize(self, text):
        clean = self.clean(text).lower()
        stopwords_en = stopwords.words("english")
        return [w for w in re.split("\W+", clean) if not w in stopwords_en]

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

tfidf = tfidf_vectorizer.fit_transform(df['Tweets'].values.astype('U'))

In [28]:
from sklearn.model_selection import train_test_split

X = df['Tweets'].values
y = df['Label'].values.astype('float')

X_train, X_test, y_train, y_test = train_test_split(tfidf, y, test_size=0.2, random_state=42)

In [29]:
from sklearn.svm import SVC
svm = SVC()

svm.fit(X_train, y_train)

SVC()

In [30]:
svm.score(X_train, y_train)

0.9468085106382979

In [31]:
svm.score(X_test, y_test)

0.6666666666666666